In [3]:
import numpy as np
from sfepy.discrete import Problem, Field, FieldVariable, Material, Integral, Equation, Equations
from sfepy.discrete.fem import Mesh, FEDomain
from sfepy.terms import Term
from sfepy.mechanics.matcoefs import stiffness_from_youngpoisson
from sfepy.solvers.ls import ScipyDirect

# Load Volume Mesh using SfePy
def load_volume_mesh(filepath):
    print("Loading volume mesh using SfePy...")
    mesh = Mesh.from_file(filepath)
    print("Volume mesh loaded successfully.")
    return mesh

# Assign Material Properties
def assign_material_properties(E, nu):
    print("Assigning material properties...")
    
    # Use stiffness_from_youngpoisson to calculate the elasticity tensor D
    D = stiffness_from_youngpoisson(dim=3, young=E, poisson=nu)
    
    # Create the material using the stiffness tensor D
    material = Material('solid', D=D)
    
    print("Material properties assigned.")
    return material

# Define the FEM problem
def setup_fem_problem(mesh, material_props):
    # Create domain and fields
    domain = FEDomain('domain', mesh)
    omega = domain.create_region('Omega', 'all')
    
    # Define displacement field
    field = Field.from_args('displacement', np.float64, 3, omega, approx_order=1)
    
    # Create field variables
    u = FieldVariable('u', 'unknown', field)
    v = FieldVariable('v', 'test', field, primary_var_name='u')
    
    # Define integral and terms
    integral = Integral('i', order=2)
    
    # Define elastic term (elasticity problem)
    elastic_term = Term.new('dw_lin_elastic(solid.D, v, u)', integral, omega, solid=material_props, v=v, u=u)
    
    # Define equation
    eq = Equation('balance_of_forces', elastic_term)
    eqs = Equations([eq])
    
    # Define the FEM problem
    problem = Problem('elasticity', equations=eqs)
    problem.time_update()

    return problem, u

# Extract global stiffness matrix (K_global)
def extract_global_stiffness_matrix(problem):
    print("Extracting global stiffness matrix...")
    
    # Assemble stiffness matrix from the problem
    K_global = problem.mtx_a  # This is the global stiffness matrix
    return K_global

# Solve the FEM problem and return the global stiffness matrix
def solve_fem_problem(problem):
    print("Solving the FEM problem...")
    problem.set_bcs()  # Apply boundary conditions if any
    
    # Solve the FEM problem
    status = problem.solve()
    
    # Extract global stiffness matrix after solving
    K_global = extract_global_stiffness_matrix(problem)
    
    print("FEM problem solved.")
    return K_global, status

# --- Main script starts here ---

# Material properties
cohesion = 0  # Cohesion in Pascals (not used directly in elasticity)
friction_angle = 10  # Friction angle in degrees (not used directly in elasticity)
density = 1500  # Density in kg/m³ (not used directly in this problem)
E = 1000  # Young's modulus in Pascals
nu = 0.3  # Poisson's ratio

# Load the volume mesh
filepath = r"C:\Users\shubh\Documents\Analytics\FOS WS\Mesh Files\mesh.vtk"
mesh = load_volume_mesh(filepath)

# Assign material properties
material_props = assign_material_properties(E, nu)

# Setup FEM problem, store intermediate terms and expressions
problem, u = setup_fem_problem(mesh, material_props)

# Solve the FEM problem and retrieve the global stiffness matrix
K_global, status = solve_fem_problem(problem)

# Print the result (global stiffness matrix)
#print("Global Stiffness Matrix (K_global):\n", K_global.toarray())  # Convert sparse matrix to array for display

Loading volume mesh using SfePy...
sfepy: reading mesh (C:\Users\shubh\Documents\Analytics\FOS WS\Mesh Files\mesh.vtk)...
sfepy:   number of vertices: 3467530
sfepy:   number of cells:
sfepy:     3_4: 6924336
sfepy: ...done in 1.09 s
Volume mesh loaded successfully.
Assigning material properties...
Material properties assigned.
sfepy: warning: bad element orientation, trying to correct...
sfepy: ...corrected
sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 1.59 s
sfepy: matrix shape: (10402590, 10402590)
sfepy: assembling matrix graph...
sfepy: ...done in 18.56 s
sfepy: matrix structural nonzeros: 342956772 (3.17e-04% fill)
Solving the FEM problem...


ValueError: you must set linear solver!